In [9]:
import random
import time
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
from selenium import webdriver
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.firefox.service import Service as FirefoxService
from bs4 import BeautifulSoup

headers = {'User-Agent': "mbambal@purdue.edu"}

def get_edgar_data_selenium(cik):
    """
    Uses Selenium with Firefox to load the SEC EDGAR page for the given CIK.
    Returns the rendered HTML.
    """
    # Set up Firefox options for headless mode
    firefox_options = FirefoxOptions()
    firefox_options.add_argument("--headless")
    firefox_options.add_argument("--no-sandbox")
    firefox_options.add_argument("--disable-dev-shm-usage")
    # Set the binary location (adjust if necessary)
    firefox_options.binary_location = "/Applications/Firefox.app/Contents/MacOS/firefox"
    
    # Create a Service object pointing to geckodriver (adjust the path if needed)
    service = FirefoxService(executable_path="/opt/homebrew/bin/geckodriver")
    
    # Initialize the Firefox WebDriver with the service and options
    driver = webdriver.Firefox(service=service, options=firefox_options)

    # Construct the SEC EDGAR URL for the given CIK (using the classic endpoint)
    url = f"https://www.sec.gov/cgi-bin/browse-edgar?CIK={cik}&owner=exclude&action=getcompany"
    driver.get(url)
    
    # Allow time for the page to load completely
    time.sleep(3)
    
    # Get the rendered HTML
    html = driver.page_source
    driver.quit()
    return html

In [10]:
def get_company_data(headers=None):
    """
    Fetches company ticker data from the SEC, converts it to a DataFrame,
    and formats the CIK with leading zeros.

    Args:
        headers (dict, optional): Optional headers for the HTTP request (e.g., User-Agent).

    Returns:
        pd.DataFrame: DataFrame containing company tickers with formatted CIKs.
    """
    url = "https://www.sec.gov/files/company_tickers.json"
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Raise error if request fails

    company_data = pd.DataFrame.from_dict(response.json(), orient='index')
    company_data['cik_str'] = company_data['cik_str'].astype(str).str.zfill(10)

    return company_data

In [11]:
def get_sic_code_from_html(html):
    """
    Parses the provided HTML using BeautifulSoup and extracts the SIC code.
    It looks for a <span> element with id="SIC" and returns its text.
    """
    soup = BeautifulSoup(html, 'html.parser')
    sic_spans = soup.find_all("span", id="SIC")
    if sic_spans:
        return sic_spans[0].get_text(strip=True)
    return None

In [12]:
def process_company(cik):
    """
    Processes a single company: fetches its EDGAR page using Firefox,
    extracts the SIC code, and returns a tuple (cik, sic_code).
    A random delay is added to avoid overwhelming the server.
    """
    try:
        html_data = get_edgar_data_selenium(cik)
        sic_code = get_sic_code_from_html(html_data)
    except Exception as e:
        print(f"Error processing CIK {cik}: {e}")
        sic_code = None
    # Add a random delay between 1 and 3 seconds
    time.sleep(random.uniform(1, 3))
    return cik, sic_code

In [13]:
def get_missing_sic_rows(all_ciks_file):
    """
    Returns a DataFrame containing rows from the input file
    where SIC_code is missing or zero.

    Args:
        all_ciks_file (str): Path to the CSV file containing cik_str, ticker, title, SIC_code.

    Returns:
        pd.DataFrame: Filtered DataFrame with missing or zero SIC_code.
    """
    df = pd.read_csv(all_ciks_file)

    # Ensure SIC_code exists and is numeric
    if 'SIC_code' not in df.columns:
        df['SIC_code'] = 0
    else:
        df['SIC_code'] = pd.to_numeric(df['SIC_code'], errors='coerce').fillna(0).astype(int)

    # Filter rows with SIC_code == 0
    df_missing = df[df['SIC_code'] == 0]

    df_missing.reset_index(drop=True, inplace=True)

    return df_missing

In [14]:
def process_batch(batch_df, max_workers=3):
    """
    Processes a batch of companies concurrently.
    Returns a list of tuples (cik, sic_code) for the companies in the batch.
    """
    results = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_cik = {executor.submit(process_company, row['cik_str']): row['cik_str'] for _, row in batch_df.iterrows()}
        for future in as_completed(future_to_cik):
            cik, sic_code = future.result()
            results.append((cik, sic_code))
    return results

In [15]:
def process_single_batch(companyData, batch_num, batch_size=1000, max_workers=3):
    """
    Processes a single batch of companies (based on the given batch number).
    Updates the companyData DataFrame with extracted SIC codes for that batch and saves the DataFrame.
    
    Args:
        companyData: The full DataFrame containing company data.
        batch_num: The batch number to process (starting from 0).
        batch_size: Number of companies to process in one batch.
        max_workers: Maximum number of threads for parallel processing.
        
    Returns:
        The updated companyData DataFrame.
    """
    start = batch_num * batch_size
    end = start + batch_size
    batch_df = companyData.iloc[start:end]
    print(f"Processing batch {batch_num} (index {start} to {end})...")
    
    batch_results = process_batch(batch_df, max_workers=max_workers)
    
    # Create a mapping dictionary from the results
    batch_map = {cik: sic for cik, sic in batch_results}
    # Map the extracted SIC codes to the corresponding rows; fill missing values with 0.
    companyData.loc[companyData.index[start:end], 'SIC_code'] = companyData.loc[companyData.index[start:end], 'cik_str'].map(batch_map).fillna(0)
    
    # Save the updated DataFrame to CSV
    companyData.to_csv(f"/Users/mayankbambal/Desktop/SIC Scraper/data/stagging/Data_with_SIC_codes{batch_num}.csv", index=False)
    print(f"Batch {batch_num} processed and saved.")
    return companyData

In [16]:
def update_sic_from_file(file1, file2):
    """
    Updates SIC_code in file1 using non-zero SIC_code values from file2,
    matching on cik_str. Only non-zero SIC_codes from file2 are applied.
    file1 is updated in-place.

    Args:
        file1 (str): Path to the main CSV file to update (will be modified).
        file2 (str): Path to the CSV file containing updated SIC_codes.
    """
    # Read both files
    df_main = pd.read_csv(file1)
    df_sic = pd.read_csv(file2)

    # Ensure cik_str is properly formatted
    df_main['cik_str'] = df_main['cik_str'].astype(str).str.zfill(10)
    df_sic['cik_str'] = df_sic['cik_str'].astype(str).str.zfill(10)

    # Keep only non-zero SIC_code rows from file2
    df_sic_nonzero = df_sic[df_sic['SIC_code'] != 0][['cik_str', 'SIC_code']]

    # Set index for fast lookup
    df_sic_nonzero.set_index('cik_str', inplace=True)

    # Update SIC_code in df_main
    def get_updated_sic(row):
        return df_sic_nonzero.loc[row['cik_str'], 'SIC_code'] \
            if row['cik_str'] in df_sic_nonzero.index else row.get('SIC_code', 0)

    df_main['SIC_code'] = df_main.apply(get_updated_sic, axis=1)

    # Save back to file1
    df_main.to_csv(file1, index=False)
    print(f"Updated SIC_code values saved to: {file1}")

In [8]:
find_sic = get_missing_sic_rows('/Users/mayankbambal/Desktop/SIC Scraper/data/final/company_tickers.csv')
find_sic.shape

NameError: name 'pd' is not defined

In [17]:
update_sic_from_file('/Users/mayankbambal/Desktop/SIC Scraper/data/final/company_tickers.csv', f"/Users/mayankbambal/Desktop/SIC Scraper/data/stagging/Data_with_SIC_codes3.csv")

Updated SIC_code values saved to: /Users/mayankbambal/Desktop/SIC Scraper/data/final/company_tickers.csv


In [ ]:
batch_number = 0  # Change this to the desired batch number
companyData = process_single_batch(find_sic, batch_number, batch_size=500, max_workers=16)
update_sic_from_file('/Users/mayankbambal/Desktop/SIC Scraper/data/final/company_tickers.csv', f"/Users/mayankbambal/Desktop/SIC Scraper/data/stagging/Data_with_SIC_codes3.csv")


Processing batch 2 (index 1000 to 1500)...
Error processing CIK 1969863: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))Error processing CIK 1674440: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error processing CIK 1392449: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

Error processing CIK 1807166: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error processing CIK 1817760: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error processing CIK 1099160: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error processing CIK 1874252: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error processing CIK 1956741: ('Connection aborted.', RemoteDisconnected('Remote end closed conn